In [12]:
import tiktoken 

In [13]:
with open("datask.txt", "r", encoding="utf-8") as f:
    text = f.read()

length = len(text)
print(length)

print("first one thousand charater")
chars = text[:1000]
print(chars)

1115394
first one thousand charater
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for

In [14]:
#all possible character
chars = set(text)
lst = sorted(list(chars))
print(lst)


vocabSize = len(lst)


['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [15]:
#Encoder and decoder
#we can use tiktoken too
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[x] for x in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hello"))
print(decode([38, 1, 62, 62, 25]))


[38, 1, 62, 62, 25]
hello


In [16]:
import torch
data = torch.Tensor(encode(lst[:1000]))
data

tensor([ 3.,  9., 21., 16., 33., 47.,  2.,  7., 27., 40., 57., 13., 42., 56.,
        17., 34., 41., 36., 61.,  4., 60., 58., 22., 52., 45., 59., 49., 53.,
        26., 19., 54., 51., 55., 39., 43., 46., 31., 28., 50., 12., 24., 64.,
        15.,  1., 18., 11., 38., 35., 29., 48., 62.,  6., 14., 25., 32., 44.,
        10.,  0., 20., 23., 37., 30.,  8., 63.,  5.])

In [ ]:
#lets make the train and test data split set kinda unsupervised learning
n = int(0.9 * len(data))
train_data = data[:n]
test_data = data[n:]

block_size = 8 #chunk of text to train the data on
train_data[:block_size+1]


tensor([ 3.,  9., 21., 16., 33., 47.,  2.,  7.])

In [20]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context}, the target is {target}")


when input is tensor([3.]), the target is 9.0
when input is tensor([3., 9.]), the target is 21.0
when input is tensor([ 3.,  9., 21.]), the target is 16.0
when input is tensor([ 3.,  9., 21., 16.]), the target is 33.0
when input is tensor([ 3.,  9., 21., 16., 33.]), the target is 47.0
when input is tensor([ 3.,  9., 21., 16., 33., 47.]), the target is 2.0
when input is tensor([ 3.,  9., 21., 16., 33., 47.,  2.]), the target is 7.0
when input is tensor([ 3.,  9., 21., 16., 33., 47.,  2.,  7.]), the target is 27.0


In [28]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x.long(), y.long()

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

print(xb.dtype)


inputs:
torch.Size([4, 8])
tensor([[34, 41, 36, 61,  4, 60, 58, 22],
        [ 7, 27, 40, 57, 13, 42, 56, 17],
        [15,  1, 18, 11, 38, 35, 29, 48],
        [ 3,  9, 21, 16, 33, 47,  2,  7]])
targets:
torch.Size([4, 8])
tensor([[41, 36, 61,  4, 60, 58, 22, 52],
        [27, 40, 57, 13, 42, 56, 17, 34],
        [ 1, 18, 11, 38, 35, 29, 48, 62],
        [ 9, 21, 16, 33, 47,  2,  7, 27]])
----
when input is [34] the target: 41
when input is [34, 41] the target: 36
when input is [34, 41, 36] the target: 61
when input is [34, 41, 36, 61] the target: 4
when input is [34, 41, 36, 61, 4] the target: 60
when input is [34, 41, 36, 61, 4, 60] the target: 58
when input is [34, 41, 36, 61, 4, 60, 58] the target: 22
when input is [34, 41, 36, 61, 4, 60, 58, 22] the target: 52
when input is [7] the target: 27
when input is [7, 27] the target: 40
when input is [7, 27, 40] the target: 57
when input is [7, 27, 40, 57] the target: 13
when input is [7, 27, 40, 57, 13] the target: 42
when input is [7, 

In [41]:


from turtle import forward
import torch
import torch.nn as nn
from torch.nn import functional as f 

class BigramlanguageModel(nn.Module):
    def __init__(self, vocabSize):
        super().__init__()
        self.tokenEmbedding_table = nn.Embedding(vocabSize, vocabSize)
    
    def forward(self, idx, targets):
        logits = self.tokenEmbedding_table(idx) #BTC

        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)


        loss = f.cross_entropy(logits, targets)
        
        return logits, loss


m = BigramlanguageModel(vocabSize=vocabSize)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.7199, grad_fn=<NllLossBackward0>)
